<a href="https://colab.research.google.com/github/stefanciprian/rd/blob/main/prices_and_volumes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('LUMN.csv')

In [2]:
df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-08-01,1.82,2.08,1.73,2.03,2.03,50755100
1,2023-08-02,1.75,1.87,1.68,1.79,1.79,39992700
2,2023-08-03,1.76,1.78,1.68,1.68,1.68,19715300
3,2023-08-04,1.67,1.77,1.64,1.66,1.66,34052900
4,2023-08-07,1.71,1.75,1.64,1.74,1.74,30195600
5,2023-08-08,1.70,1.75,1.67,1.71,1.71,20832800
6,2023-08-09,1.70,1.78,1.68,1.74,1.74,17619100
7,2023-08-10,1.75,2.02,1.75,1.89,1.89,47410600
8,2023-08-11,1.88,1.91,1.82,1.86,1.86,16540600
9,2023-08-14,1.84,1.86,1.77,1.81,1.81,17572900
